In [13]:
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd
from sklearn import datasets
from sklearn.metrics import accuracy_score,f1_score,recall_score,roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import plot_tree
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
import xgboost as xgb
from clearml import Task
from sklearn.compose import ColumnTransformer
import json

In [2]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=7X93ISKOF6CVEQC0JPAR
%env CLEARML_API_SECRET_KEY=yFGbjolk8W5d9dtYYWEBX4KDXKZIcciDuNd10UGlOVoNLFpPJs

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=7X93ISKOF6CVEQC0JPAR
env: CLEARML_API_SECRET_KEY=yFGbjolk8W5d9dtYYWEBX4KDXKZIcciDuNd10UGlOVoNLFpPJs


In [3]:
task = Task.init(project_name='mlops', task_name='XGBoost prob1_phase1')


ClearML Task: created new task id=b0c3460321e14152b37e88a2ab5be3ec
2023-06-25 15:06:15,101 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2023-06-25 15:06:15,105 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/acbdbb6d90ce4bc798c78897b4e4c353/experiments/b0c3460321e14152b37e88a2ab5be3ec/output/log


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd4d9783400>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/events.add_batch
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd4d9752d10>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/events.add_batch
Retrying (Retry(total=236, connect=236, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd4d9783880>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/events.add_batch


In [5]:
df=pd.read_parquet('raw_train.parquet')

In [6]:
X=df.drop('label',axis=1)
Y=df['label']
X_train,Y_train,X_test,Y_test=train_test_split(X,Y,test_size=0.1,random_state=42)


In [ ]:
print(X_train.head())

In [15]:
with open('features_config.json','r') as f:
    features_config=json.load(f)
features_config

{'numeric_columns': ['feature3',
  'feature4',
  'feature5',
  'feature6',
  'feature7',
  'feature8',
  'feature9',
  'feature10',
  'feature11',
  'feature12',
  'feature13',
  'feature14',
  'feature15',
  'feature16'],
 'category_columns': ['feature2', 'feature1'],
 'target_column': 'label',
 'ml_type': 'classification'}

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class DropColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns = []):
        self.columns = columns
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Perform arbitary transformation
        X = X.drop(columns=self.columns)
        return X

In [16]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), 
           ("scaler", StandardScaler()),
           ('pca', PCA())
           ]
)
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        # ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, features_config['numeric_columns']),
        ("cat", categorical_transformer, features_config['category_columns']),
    ]
)

Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd443477bb0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/tasks.get_all
Retrying (Retry(total=236, connect=236, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd443432aa0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v2.23/tasks.get_all


In [ ]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations

In [ ]:
try:
    # try to load a model
    bst = xgb.Booster(params=param, model_file='xgb.01.model')
    bst.load_model('xgb.01.model')
except Exception:
    bst = None

# if we dont have one train a model
if bst is None:
    bst = xgb.train(param, dtrain, num_round)